In [82]:
import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split

from scipy import stats

from sklearn.neighbors import KNeighborsClassifier

pd.options.mode.chained_assignment = None

In [83]:
nomes_col = ['Class',
             'Alcohol', 
             'Malic_acid',
             'Ash', 
             'Alcalinity_of_ash', 
             'Magnesium', 
             'Total_phenols',
             'Flavanoids',
             'Nonflavanoid_phenols',
             'Proanthocyanins',
             'Color_intensity',
             'Hue',
             'AOD280_OD315_of_diluted_wines',
             'Proline',
            ]


In [84]:
df = pd.read_csv("data/wine.data", names=nomes_col)


df_zero_um = df.copy()

In [85]:
def divide_por_dezmil(coluna):
    for x in range(coluna.size):
        coluna[x] = coluna[x]/10000
    return coluna

def divide_por_mil(coluna):
    for x in range(coluna.size):
        coluna[x] = coluna[x]/1000
    return coluna

def divide_por_cem(coluna):
    for x in range(coluna.size):
        coluna[x] = coluna[x]/100
    return coluna

def divide_por_dez(coluna):
    for x in range(coluna.size):
        coluna[x] = coluna[x]/10
    return coluna    

In [86]:
df_zero_um.Proline = divide_por_dezmil(df_zero_um.Proline)
df_zero_um.Magnesium = divide_por_mil(df_zero_um.Magnesium)
df_zero_um.Alcalinity_of_ash = divide_por_cem(df_zero_um.Alcalinity_of_ash)
df_zero_um.Alcohol = divide_por_cem(df_zero_um.Alcohol)
df_zero_um.Malic_acid = divide_por_dez(df_zero_um.Malic_acid)
df_zero_um.Ash = divide_por_dez(df_zero_um.Ash)
df_zero_um.Total_phenols = divide_por_dez(df_zero_um.Total_phenols)
df_zero_um.Flavanoids = divide_por_dez(df_zero_um.Flavanoids)
df_zero_um.Proanthocyanins = divide_por_dez(df_zero_um.Proanthocyanins)
df_zero_um.Color_intensity = divide_por_dez(df_zero_um.Color_intensity)
df_zero_um.Hue = divide_por_dez(df_zero_um.Hue)
df_zero_um.AOD280_OD315_of_diluted_wines= divide_por_dez(df_zero_um.AOD280_OD315_of_diluted_wines)


In [87]:
y_df = df.pop("Class")

y_zero_um = df_zero_um.pop("Class")

In [88]:
def calcular_taxas_para_knn(k, qtd_taxas, porcent_test, atributos, classe):
    array_taxas = []

    for n in range(k):
        arrays = []
        for i in range(qtd_taxas):
            x_treino, x_teste, y_treino, y_teste = train_test_split(atributos, classe,test_size=porcent_test, random_state=i)
            classificador = KNeighborsClassifier(n_neighbors=n+1)
            classificador.fit(x_treino, y_treino)
            taxas = classificador.score(x_teste, y_teste)
            arrays.append(taxas)

        array_taxas.append(arrays)
        
    return array_taxas

array_taxas_df = calcular_taxas_para_knn(1, 100, 0.5, df, y_df)

array_taxas_zero_um = calcular_taxas_para_knn(1, 100, 0.5, df_zero_um, y_zero_um)

In [89]:
def calcular_intervalo_confianca(hash_medias, hash_diferencas_total):
    for i in range(len(hash_medias)):
        for x in range(len(hash_medias[i+1])):
            
            desvio_padrao = np.std(hash_diferencas_total[i+1][x])
            
            erro_padrao = desvio_padrao/np.sqrt(len(hash_diferencas_total[i+1][x]))

            multiplicador = abs(stats.distributions.norm.ppf(0.025))

            print(i+1, (hash_medias[i+1][x] - multiplicador*erro_padrao, hash_medias[i+1][x] + multiplicador*erro_padrao)
            )

In [90]:
def calcular_medias_separadas(array):

    hash_saida = {}
    array_aux = []
    for x in range(len(array)):  
        array_aux.append(np.mean(array[x]))
        hash_saida[x+1] = array_aux
        array_aux = []
    return hash_saida

medias_separadas = calcular_medias_separadas(array_taxas_df)

medias_separadas_zero_um = calcular_medias_separadas(array_taxas_zero_um)

In [91]:
hash_taxas = {}
array_aux = []
for x in range(len(array_taxas_df)):
    array_aux.append(array_taxas_df[x])
    hash_taxas[x+1] = array_aux
    array_aux = []

hash_taxas2 = {}
array_aux2 = []
for x in range(len(array_taxas_zero_um)):
    array_aux2.append(array_taxas_zero_um[x])
    hash_taxas2[x+1] = array_aux2
    array_aux2 = []

calcular_intervalo_confianca(medias_separadas, hash_taxas)
calcular_intervalo_confianca(medias_separadas_zero_um, hash_taxas2)

1 (0.7083788816487575, 0.7266772981265237)
1 (0.917809882487535, 0.9278080950405551)
